In [1]:
import numpy as np
import pandas as pd
import datetime
import copy
import time
import os
import re
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import operator

from tqdm.auto import tqdm, trange
from tqdm.notebook import tqdm
from datetime import timedelta

tqdm.pandas()

In [3]:
# Edit to point to your MIMIC directory.
dataDirStr = '/Users/gmessier/data/mimic-1.4/'

In [5]:
labevents_df = pd.read_csv(dataDirStr + "LABEVENTS.csv")
labevents_df.columns = labevents_df.columns.str.lower()
labevents_df

,row_id,subject_id,hadm_id,itemid,charttime,value,valuenum,valueuom,flag
0,281,3,NaN,50820,2101-10-12 16:07:00,7.39,7.39,units,NaN
1,282,3,NaN,50800,2101-10-12 18:17:00,ART,NaN,NaN,NaN
2,283,3,NaN,50802,2101-10-12 18:17:00,-1,-1.00,mEq/L,NaN
3,284,3,NaN,50804,2101-10-12 18:17:00,22,22.00,mEq/L,NaN
4,285,3,NaN,50808,2101-10-12 18:17:00,0.93,0.93,mmol/L,abnormal
...,...,...,...,...,...,...,...,...,...
27854050,27428435,96443,103219.0,50882,2109-12-30 01:40:00,26,26.00,mEq/L,NaN
27854051,27428436,96443,103219.0,50885,2109-12-30 01:40:00,2.1,2.10,mg/dL,abnormal
27854052,27428437,96443,103219.0,50902,2109-12-30 01:40:00,97,97.00,mEq/L,NaN
27854053,27428438,96443,103219.0,50911,2109-12-30 01:40:00,2,2.00,ng/mL,NaN


`LABEVENTS` contains all laboratory measurements for a given patient, including out patient data. The time associated with each entry is at the time of fluid acquisition.

In [6]:
print(f"There are {labevents_df.subject_id.nunique()} patients who have taken a lab measurement")

There are 46252 patients who have taken a lab measurement


`itemid` is the identifier for a single measurment type in the database. Refer to `D_LABITEMS` table for exact definitions of each `itemid`.

In [7]:
print(f"There are {labevents_df.itemid.nunique()} different types of measurements")

There are 726 different types of measurements


In [8]:
c = labevents_df.itemid.value_counts()[:5]
p = labevents_df.itemid.value_counts(normalize=True).mul(100).round(2)[:5]
pd.concat([c,p], axis=1, keys=['counts', '%'])

,counts,%
51221,881764,3.17
50971,845737,3.04
50983,808401,2.90
50912,797389,2.86
50902,795480,2.86


`value` and `valuenum` contain the value measured for the given `itemid`. If `value` is numeric, then `valuenum` is the same value. If `value` is not numeric, then `valuenum` will be null.

`flag` indicates whether the given `value` for the `itemid` is considered abnormal or not (using predefined thresholds)

In [9]:
c = labevents_df.flag.value_counts()
p = labevents_df.flag.value_counts(normalize=True).mul(100).round(2)
pd.concat([c,p], axis=1, keys=['counts', '%'])

,counts,%
abnormal,9779594,99.34
delta,64572,0.66
